# 1 MIP Bellman

## 1.1 DRMDP with decision dependent ambiguity set (DRMDP DD1)

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

def solve_bellman_formulation1(act_dim, num_states, lam, next_V, k, delta_s, rho_s, delta_0_s, rho_0_s, ub_a):
    model2 = gp.Model()
    model2.setParam('OutputFlag', False)
    a = [0 for i in range(act_dim)]
    w = [0 for i in range(num_states)]
    u = [0 for i in range(num_states)]
    m0 = [[0 for j in range(num_states)] for i in range(act_dim)]
    m1 = [[0 for j in range(num_states)] for i in range(act_dim)]
    r = model2.addVar(vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = GRB.INFINITY, name = 'r')
    q = model2.addVar(vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = GRB.INFINITY, name = 'q')
    for i in range(act_dim):
        a[i] = model2.addVar(vtype=GRB.INTEGER, lb = 0.0, ub = ub_a[i], name = 'a%d' %i)
    for i in range(num_states):
        w[i] = model2.addVar(vtype=GRB.CONTINUOUS, lb = 0.0, ub = k[i], name = 'w%d' %i)
    for i in range(num_states):
        u[i] = model2.addVar(vtype=GRB.CONTINUOUS, lb = 0.0, ub = k[i], name = 'u%d' %i)
    for i in range(act_dim):
        for j in range(num_states):
            m0[i][j] = model2.addVar(vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = GRB.INFINITY, name = 'm0-%d%d' %(i,j))
    for i in range(act_dim):
        for j in range(num_states):
                m1[i][j] = model2.addVar(vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = GRB.INFINITY, name = 'm1-%d%d' %(i,j))

    # Objective
    obj = 1*r + delta_0_s
    for i in range(act_dim):
        obj.addTerms(delta_s[i], a[i])
    for j in range(num_states):
        obj.addTerms(-rho_0_s[j], w[j])
    for i in range(act_dim):
        for j in range(num_states):
            obj.addTerms(-rho_s[i][j], m0[i][j])
    for j in range(num_states):
        obj.addTerms(rho_0_s[j], u[j])
    for i in range(act_dim):
        for j in range(num_states):
            obj.addTerms(rho_s[i][j], m1[i][j])
    model2.setObjective(obj, GRB.MAXIMIZE)   

    # Constraint 1
    model2.addConstr(q-r, GRB.GREATER_EQUAL, 0, "c1");

    # Constraint 2
    for j in range(num_states):
        model2.addConstr(lam*next_V[j] + w[j] - u[j] - q >= 0)
    
    # McCormick envelopes
    # M0
    for i in range(act_dim):
        for j in range(num_states):
            model2.addConstr(m0[i][j] - 0*w[j] - a[i]*0 + 0*0 >= 0, "m0-1-%d%d" %(i,j))
            model2.addConstr(m0[i][j] - ub_a[i]*w[j] - a[i]*k[j] + ub_a[i]*k[j] >= 0, "m0-2-%d%d" %(i,j))
            model2.addConstr(m0[i][j] - ub_a[i]*w[j] - a[i]*0 + ub_a[i]*0 <= 0, "m0-3-%d%d" %(i,j))
            model2.addConstr(m0[i][j] - a[i]*k[j] - 0*w[j] + 0*k[j] <= 0, "m0-4-%d%d" %(i,j))
    
    # M1
    for i in range(act_dim):
        for j in range(num_states):
            model2.addConstr(m1[i][j] - 0*u[j] - a[i]*0 + 0*0 >= 0, "m1-1-%d%d" %(i,j))
            model2.addConstr(m1[i][j] - ub_a[i]*u[j] - a[i]*k[j] + ub_a[i]*k[j] >= 0, "m1-2-%d%d" %(i,j))
            model2.addConstr(m1[i][j] - ub_a[i]*u[j] - a[i]*0 + ub_a[i]*0 <= 0, "m1-3-%d%d" %(i,j))
            model2.addConstr(m1[i][j] - a[i]*k[j] - 0*u[j] + 0*k[j] <= 0, "m1-4-%d%d" %(i,j))

    result = model2.optimize()
    a = []
    for v in model2.getVars():
        # print('%s %g,' % (v.varName, v.x), end = " ")
        if 'a' in v.varName:
            a.append(v.x)
    optimal_objective = model2.getObjective().getValue()
    optimal_a = np.asarray(a)
    return optimal_objective, optimal_a

## 1.2 DRMDP with decision dependent ambiguity set (DRMDP DD2)

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

def solve_bellman_formulation2(act_dim, num_states, lam, next_V, k, K, delta_s, rho_s, sigma_s,
                                            delta_0_s, rho_0_s, sigma_0_s, bar_Sigma_s, ub_a):
    model1 = gp.Model()
    model1.setParam('OutputFlag', False)
    a = [0 for i in range(act_dim)]
    w = [0 for i in range(num_states)]
    u = [0 for i in range(num_states)]
    Q = [[0 for j in range(num_states)] for i in range(num_states)]
    h = [0 for i in range(num_states)]
    m0 = [[0 for j in range(num_states)] for i in range(act_dim)]
    m1 = [[0 for j in range(num_states)] for i in range(act_dim)]
    m2 = [[[0 for j_ in range(num_states)] for j in range(num_states)] for i in range(act_dim)]
    m3 = [[[[0 for j_ in  range(num_states)] for j in range(num_states)] for i_ in range(act_dim)] for i in range(act_dim)]
    m4 = [[0 for j in range(num_states)] for i in range(num_states)]
    m5 = [[0 for j in range(num_states)] for i in range(num_states)]


    r = model1.addVar(vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = GRB.INFINITY, name = 'r')
    q = model1.addVar(vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = GRB.INFINITY, name = 'q')
    for i in range(act_dim):
        a[i] = model1.addVar(vtype=GRB.INTEGER, lb = 0.0, ub = ub_a[i], name = 'a%d' %i)
    for i in range(num_states):
        w[i] = model1.addVar(vtype=GRB.CONTINUOUS, lb = 0.0, ub = k[i], name = 'w%d' %i)
    for i in range(num_states):
        u[i] = model1.addVar(vtype=GRB.CONTINUOUS, lb = 0.0, ub = k[i], name = 'u%d' %i)
    for i in range(num_states):
        for j in range(num_states):
            Q[i][j] = model1.addVar(vtype=GRB.CONTINUOUS, lb = -np.sqrt(K[i][i]*K[j][j]), ub = np.sqrt(K[i][i]*K[j][j]), name = 'Q%d%d' %(i,j))
    for i in range(num_states):
        h[i] = model1.addVar(vtype=GRB.CONTINUOUS, name = 'h%d' %i)
    for i in range(act_dim):
        for j in range(num_states):
            m0[i][j] = model1.addVar(vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = GRB.INFINITY, name = 'm0-%d%d' %(i,j))
    for i in range(act_dim):
        for j in range(num_states):
            m1[i][j] = model1.addVar(vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = GRB.INFINITY, name = 'm1-%d%d' %(i,j))
    for i in range(act_dim):
        for j in range(num_states):
            for j_ in range(num_states):
                m2[i][j][j_] = model1.addVar(vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = GRB.INFINITY, name = 'm2-%d%d%d' %(i,j,j_))
    for i in range(act_dim):
        for i_ in range(act_dim):
            for j in range(num_states):
                for j_ in range(num_states):
                    m3[i][i_][j][j_] = model1.addVar(vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = GRB.INFINITY, name = 'm3-%d%d%d%d' %(i,i_,j,j_))
    for i in range(num_states):
        for j in range(num_states):
            m4[i][j] = model1.addVar(vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = GRB.INFINITY, name = 'm4-%d%d' %(i,j))
    for i in range(num_states):
        for j in range(num_states):
            m5[i][j] = model1.addVar(vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = GRB.INFINITY, name = 'm5-%d%d' %(i,j))
    
    # Objective
    obj = 1*r + delta_0_s
    for i in range(act_dim):
        obj.addTerms(delta_s[i], a[i])
    for j in range(num_states):
        obj.addTerms(-rho_0_s[j], w[j])
    for j in range(num_states):
        for i in range(act_dim):
            obj.addTerms(-rho_s[i][j], m0[i][j])
    for j in range(num_states):
        obj.addTerms(rho_0_s[j], u[j])
    for j in range(num_states):
        for i in range(act_dim):
            obj.addTerms(rho_s[i][j], m1[i][j])
    for j in range(num_states):
        for j_ in range(num_states):
            obj.addTerms(rho_0_s[j]*rho_0_s[j_], Q[j][j_])
    for j in range(num_states):
        for j_ in range(num_states):
            for i in range(act_dim):
                obj.addTerms(rho_0_s[j_]*rho_s[i][j] + rho_0_s[j]*rho_s[i][j_], m2[i][j][j_])
    for j in range(num_states):
        for j_ in range(num_states):
            for i in range(act_dim):
                for i_ in range(act_dim):
                    obj.addTerms(rho_s[i][j]*rho_s[i_][j_], m3[i][i_][j][j_])
    for j in range(num_states):
        for j_ in range(num_states):
            obj.addTerms(-bar_Sigma_s[j][j_]*sigma_0_s, Q[j][j_])
    for j in range(num_states):
        for j_ in range(num_states):
            for i in range(act_dim):
                obj.addTerms(-bar_Sigma_s[j][j_]*sigma_s[i], m2[i][j][j_])
    model1.setObjective(obj, GRB.MAXIMIZE)

    # Constraint 1
    constraint1 = q - r
    for i in range(num_states):
        for j in range(num_states):
            constraint1.addTerms(-1,m5[i][j])
    model1.addConstr(constraint1, GRB.GREATER_EQUAL, 0, "c1")

    # Constraint 2 -- j constraints
    for j in range(num_states):
        constraint = lam*next_V[j] + w[j] - u[j] - q
        for i in range(num_states): 
            constraint.addTerms(1, m4[i][j])
        for j_ in range(num_states):
            constraint.addTerms(-2*rho_0_s[j_], Q[j][j_])
        for j_ in range(num_states):
            for i in range(act_dim):
                constraint.addTerms(-2*rho_s[i][j_], m2[i][j][j_])
        model1.addConstr(constraint, GRB.GREATER_EQUAL, 0, "c2-%d" %j);

    # PSD
    for i in range(num_states):
        model1.addConstr(Q[i][i] >= gp.quicksum(Q[i][j] for j in range(num_states)), 'PSD diagonal dominance')
        model1.addConstr(Q[i][i] >= 0, "PSD positive diagonal")
        
    # McCormick envelopes
    # M0
    for i in range(act_dim):
        for j in range(num_states):
            model1.addConstr(m0[i][j] - 0*w[j] - a[i]*0 + 0*0 >= 0, "m0-1-%d%d" %(i,j))
            model1.addConstr(m0[i][j] - ub_a[i]*w[j] - a[i]*k[j] + ub_a[i]*k[j] >= 0, "m0-2-%d%d" %(i,j))
            model1.addConstr(m0[i][j] - ub_a[i]*w[j] - a[i]*0 + ub_a[i]*0 <= 0, "m0-3-%d%d" %(i,j))
            model1.addConstr(m0[i][j] - a[i]*k[j] - 0*w[j] + 0*k[j] <= 0, "m0-4-%d%d" %(i,j))
    
    # M1
    for i in range(act_dim):
        for j in range(num_states):
            model1.addConstr(m1[i][j] - 0*u[j] - a[i]*0 + 0*0 >= 0, "m1-1-%d%d" %(i,j))
            model1.addConstr(m1[i][j] - ub_a[i]*u[j] - a[i]*k[j] + ub_a[i]*k[j] >= 0, "m1-2-%d%d" %(i,j))
            model1.addConstr(m1[i][j] - ub_a[i]*u[j] - a[i]*0 + ub_a[i]*0 <= 0, "m1-3-%d%d" %(i,j))
            model1.addConstr(m1[i][j] - a[i]*k[j] - 0*u[j] + 0*k[j] <= 0, "m1-4-%d%d" %(i,j))
    # M2
    for i in range(act_dim):
        for j in range(num_states):
            for j_ in range(num_states):
                model1.addConstr(m2[i][j][j_] - 0*Q[j][j_] - a[i]*(-np.sqrt(K[j][j]*K[j_][j_])) + 0*(-np.sqrt(K[j][j]*K[j_][j_])) >= 0, "m2-1-%d%d%d" %(i,j,j_))
                model1.addConstr(m2[i][j][j_] - ub_a[i]*Q[j][j_] - a[i]*np.sqrt(K[j][j]*K[j_][j_]) + ub_a[i]*np.sqrt(K[j][j]*K[j_][j_]) >= 0, "m2-2-%d%d%d" %(i,j,j_))
                model1.addConstr(m2[i][j][j_] - ub_a[i]*Q[j][j_] - a[i]*(-np.sqrt(K[j][j]*K[j_][j_])) + ub_a[i]*(-np.sqrt(K[j][j]*K[j_][j_])) <= 0, "m2-3-%d%d%d" %(i,j,j_))
                model1.addConstr(m2[i][j][j_] - a[i]*np.sqrt(K[j][j]*K[j_][j_]) - 0*Q[j][j_] + 0*np.sqrt(K[j][j]*K[j_][j_]) <= 0, "m2-4-%d%d%d" %(i,j,j_))

    # ub-m2 -- ub_a[i]*np.sqrt(K[j][j]*K[j_][j_])
    # lb-m2 -- -ub_a[i]*np.sqrt(K[j][j]*K[j_][j_])

    # M3
    for i in range(act_dim):
        for i_ in range(act_dim):
            for j in range(num_states):
                for j_ in range(num_states):
                    model1.addConstr(m3[i][i_][j][j_] - 0*m2[i][j][j_] - a[i_]*(-ub_a[i]*np.sqrt(K[j][j]*K[j_][j_])) + 0*(-ub_a[i]*np.sqrt(K[j][j]*K[j_][j_])) >= 0)
                    model1.addConstr(m3[i][i_][j][j_] - ub_a[i_]*m2[i][j][j_] - a[i_]*(ub_a[i]*np.sqrt(K[j][j]*K[j_][j_])) + ub_a[i_]*(ub_a[i]*np.sqrt(K[j][j]*K[j_][j_])) >= 0)
                    model1.addConstr(m3[i][i_][j][j_] - ub_a[i_]*m2[i][j][j_] - a[i_]*(-ub_a[i]*np.sqrt(K[j][j]*K[j_][j_])) + ub_a[i_]*(-ub_a[i]*np.sqrt(K[j][j]*K[j_][j_])) <= 0)
                    model1.addConstr(m3[i][i_][j][j_] - a[i_]*(ub_a[i]*np.sqrt(K[j][j]*K[j_][j_])) - 0*m2[i][j][j_] + 0*(ub_a[i]*np.sqrt(K[j][j]*K[j_][j_])) <= 0)


    for i in range(num_states):
        for j in range(num_states):
            model1.addConstr(m4[i][j] <= 1)
            model1.addConstr(m4[i][j] >= -1)
            model1.addConstr(m5[i][j] <= 1)
            model1.addConstr(m5[i][j] >= -1)
        
        

    result = model1.optimize()
    a = []
    for v in model1.getVars():
        # print('%s %g,' % (v.varName, v.x), end = " ")
        if 'a' in v.varName:
            a.append(v.x)
    optimal_objective = model1.getObjective().getValue()
    optimal_a = np.asarray(a)
    return optimal_objective, optimal_a

## 1.3 Regular MDP

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

def solve_bellman_formulation3(act_dim, num_states, lam, next_V, delta_s, rho_s, delta_0_s, rho_0_s, ub_a):
    model3 = gp.Model()
    model3.setParam('OutputFlag', False)
    a = [0 for i in range(act_dim)]
    for i in range(act_dim):
        a[i] = model3.addVar(vtype=GRB.INTEGER, lb = 0.0, ub = ub_a[i], name = 'a%d' %i)
    # Objective
    obj = delta_0_s + gp.quicksum(delta_s[i]*a[i] for i in range(act_dim))
    for j in range(num_states):
        obj += lam*rho_0_s[j]*next_V[j]
    for i in range(act_dim):
        for j in range(num_states):
            obj.addTerms(lam*rho_s[i][j]*next_V[j], a[i])
    model3.setObjective(obj, GRB.MAXIMIZE)
    
    result = model3.optimize()
    a = []
    for v in model3.getVars():
        # print('%s %g,' % (v.varName, v.x), end = " ")
        if 'a' in v.varName:
            a.append(v.x)
    optimal_objective = model3.getObjective().getValue()
    optimal_a = np.asarray(a)
    return optimal_objective, optimal_a

## 1.4 Test

In [ ]:
import numpy as np

# population size
N = 10
M = 1

act_dim = 2
num_states = (N+1)**2
lam = 0.9
next_V = np.zeros(num_states)


# pernalty coefficients
k = np.ones(num_states) 
K = np.ones((num_states, num_states))

# LDR coefficients
delta_s = np.ones(act_dim)
delta_0_s = 1
sigma_s = np.ones(act_dim)
sigma_0_s = 1
rho_s = np.ones((act_dim, num_states))
rho_0_s = np.ones(num_states)
ps = np.ones(num_states)

bar_Sigma_s = np.ones((num_states, num_states))

# bound of actions
ub_a = np.asarray([N,M])

objective, a1 = solve_bellman_formulation1(act_dim, num_states, lam, next_V, k, delta_s, rho_s, delta_0_s, rho_0_s, ub_a)
print(objective)
print(a1)


objective, a2 = solve_bellman_formulation2(act_dim, num_states, lam, next_V, k, K, delta_s, rho_s, sigma_s,
                                            delta_0_s, rho_0_s, sigma_0_s, bar_Sigma_s, ub_a)
print(objective)
print(a2)

objective, a3 = solve_bellman_formulation3(act_dim, num_states, lam, next_V, delta_s, rho_s, delta_0_s, rho_0_s, ub_a)
print(objective)
print(a3)

# 2 Vaccine Model Simulation

In [ ]:
import numpy as np
import scipy.stats as stats
from sklearn.linear_model import LinearRegression

def get_phi(state, action, population_size, trans_reduc_types, tau, mu):
    # tau: probability that a susceptible person becomes infected upon contact with an infectious individual 
    # mu: the rate of contacts (contacts occur according to a homogenous Poisson process, the rate of contacts)
    # beta(t): probability that the next interaction of a random susceptible person is with an infectious person
    # alpha(t): the fractional reduction in the infection transmission rate
    N = population_size
    M = trans_reduc_types
    xst = state[0]
    xit = state[1]
    yvt = action[0]
    yTt = action[1]
    beta_t = xit/N
    alpha_t = yTt/(M+1)
    phi_t = 1 - np.exp(-(1-alpha_t)*mu*beta_t*tau)
    return phi_t
    

def get_tilde_pas(state, action, population_size, trans_reduc_types, discretize, discretized_N, tau, mu):
    # given a state, an action, return estimate of P(*|a,s)
    N = population_size
    M = trans_reduc_types
    dicretize_level = np.int(N/discretized_N)
    xst = state[0]
    xit = state[1]
    yvt = action[0]
    yTt = action[1]
    phi_t = get_phi(state, action, N, M, tau, mu)
    if discretize == False:
        pas_matrix = np.zeros((N+1, N+1))
        for xs in range(N+1):
            for xi in range(N+1):
                if xs + xi == xst - yvt:
                    pas_matrix[xs][xi] = stats.binom.pmf(xi, xst - yvt, phi_t)
        pas = pas_matrix.flatten()
    else:
        pas_matrix = np.zeros((discretized_N+1, discretized_N+1))
        for s0 in range(discretized_N+1):
            for s1 in range(discretized_N+1):
                if s0 + s1 == np.int((xst - yvt)/dicretize_level):
                    pas_matrix[s0][s1] = stats.binom.pmf(s1*dicretize_level, xst - yvt, phi_t)
        pas = pas_matrix.flatten()
        pas = pas/np.sum(pas)
    return pas
        

def get_tilde_ras(state, action, population_size, trans_reduc_types, 
               cT_multiplier, wtp, cost_per_infect, qaly_loss_per_infect, vaccine_price, tau, mu):
    # given a state, an action, return estimate of r(a,s)
    N = population_size
    M = trans_reduc_types
    xst = state[0]
    xit = state[1]
    yvt = action[0]
    yTt = np.int(action[1])
    phi_t = get_phi(state, action, N, M, tau, mu)
    E_it = (xst - yvt)*phi_t
    # cost of implementing transmission reducing method 
    c_T = np.arange(M+1)*cT_multiplier
    # policy maker WTP for health  
    lam = wtp
    c = cost_per_infect
    w = qaly_loss_per_infect
    p = vaccine_price
    r = -lam*w*E_it - c*E_it - c_T[yTt] - p*yvt
    return r

def get_bar_pas_coefficients(state, population_size, trans_reduc_types, discretize = False, discretized_N = 100, tau = 0.8, mu = 1, vaccine_limit = 100):
    N = population_size
    M = trans_reduc_types
    if discretize == False:
        num_states = (N+1)**2
    else:
        num_states = (discretized_N+1)**2
    xst = state[0]
    xit = state[1]
    num_actions = 4
    ds_action = np.zeros((num_actions, 2))
    ds_pas = np.zeros((num_actions, num_states))
    i = 0
    yv_bound = [0, np.min([vaccine_limit, xst])]
    yT_bound = [0, M]
    for yv in yv_bound:
        for yT in yT_bound:
            action = np.asarray([yv, yT])
            ds_action[i,:] = action
            ds_pas[i,:] = get_tilde_pas(state, action, N, M, discretize, discretized_N, tau, mu)
            i += 1
    reg = LinearRegression().fit(ds_action, ds_pas)
    rho_0_s = reg.intercept_
    rho_s = reg.coef_
    return rho_0_s, np.transpose(rho_s)

def get_bar_pas(action, population_size, act_dim, rho_0_s, rho_s, discretize = False, discretized_N = 100):
    N = population_size
    if discretize == False:
        num_states = (N+1)**2
    else:
        num_states = (discretized_N+1)**2
    pas = rho_0_s
    for j in range(num_states):
        for i in range(act_dim):
            pas[j] += rho_s[i][j]*action[i]
    return pas

def get_far_from_bar_pas(action, population_size, act_dim, rho_0_s, rho_s, discretize = False, discretized_N = 100):
    bar_pas = get_bar_pas(action, population_size, act_dim, rho_0_s, rho_s, discretize, discretized_N)
    ind = np.argpartition(bar_pas, -10)[-10:]
    largest_10_values = bar_pas[ind].copy()
    np.random.shuffle(largest_10_values)
    far_from_bar_pas = bar_pas.copy()
    far_from_bar_pas[ind] = largest_10_values
    return far_from_bar_pas

def get_far2_from_bar_pas(action, population_size, act_dim, rho_0_s, rho_s, discretize = False, discretized_N = 100):
    bar_pas = get_bar_pas(action, population_size, act_dim, rho_0_s, rho_s, discretize, discretized_N)
    ind = np.argpartition(bar_pas, -2)[-2:]
    largest_2_values = bar_pas[ind].copy()
    largest_2_values_copy = largest_2_values.copy()
    largest_2_values_copy[0] = largest_2_values[1]
    largest_2_values_copy[1] = largest_2_values[0]
    far2_from_bar_pas = bar_pas.copy()
    far2_from_bar_pas[ind] = largest_2_values_copy
    return far2_from_bar_pas
    
def get_ras_coefficients(state, population_size, trans_reduc_types, cT_multiplier = 2, wtp = 1, 
                     cost_per_infect = 1, qaly_loss_per_infect = 1, vaccine_price = 0.1, tau = 0.8, mu = 1, vaccine_limit = 100):
    N = population_size
    M = trans_reduc_types
    xst = state[0]
    xit = state[1]
    num_actions = 4
    ds_action = np.zeros((num_actions, 2))
    ds_reward = np.zeros(num_actions)
    i = 0
    yv_bound = [0, np.min([vaccine_limit, xst])]
    yT_bound = [0, M]
    for yv in yv_bound:
        for yT in yT_bound:
            action = np.asarray([yv, yT])
            ds_action[i,:] = action
            ds_reward[i] = get_tilde_ras(state, action, N, M, cT_multiplier, wtp, 
                                                cost_per_infect, qaly_loss_per_infect, vaccine_price, tau, mu)
            i += 1
    reg = LinearRegression().fit(ds_action, ds_reward)
    delta_0_s = reg.intercept_
    delta_s = reg.coef_
    return delta_0_s, delta_s

def get_ras(action, delta_s, delta_0_s):
    ras = delta_0_s
    for i in range(len(action)):
        ras += delta_s[i]*action[i]
    return ras

def get_max_tilde_ras(state, population_size, trans_reduc_types, cT_multiplier = 2, wtp = 1, 
                     cost_per_infect = 1, qaly_loss_per_infect = 1, vaccine_price = 0.1, tau = 0.8, mu = 1, vaccine_limit = 100):
    # max_a {tilde_ras} for a given state
    N = population_size
    M = trans_reduc_types
    xst = state[0]
    xit = state[1]
    yv_limit = np.int(np.min([vaccine_limit, xst]))
    num_actions = (yv_limit+1)*(M+1)
    ds_action = np.zeros((num_actions, 2))
    ds_reward = np.zeros(num_actions)
    i = 0
    for yv in range(yv_limit+1):
        for yT in range(M+1):
            action = np.asarray([yv, yT])
            ds_action[i,:] = action
            ds_reward[i] = get_tilde_ras(state, action, N, M, cT_multiplier, wtp, 
                                                cost_per_infect, qaly_loss_per_infect, vaccine_price, tau, mu)
            i += 1
    max_ras = np.max(ds_reward)
    max_idx = np.argmax(ds_reward)
    return ds_action[max_idx], max_ras

def get_max_ras(state, trans_reduc_types, delta_s, delta_0_s, vaccine_limit = 100):
    # max_a {ras} for a given state
    M = trans_reduc_types
    xst = state[0]
    xit = state[1]
    yv_limit = np.int(np.min([vaccine_limit, xst]))
    num_actions = (yv_limit+1)*(M+1)
    ds_action = np.zeros((num_actions, 2))
    ds_reward = np.zeros(num_actions)
    i = 0 
    for yv in range(yv_limit+1):
        for yT in range(M+1):
            action = np.asarray([yv, yT])
            ds_action[i,:] = action
            ds_reward[i] = get_ras(action, delta_s, delta_0_s)
            i += 1
    max_ras = np.max(ds_reward)
    max_idx = np.argmax(ds_reward)
    return ds_action[max_idx], max_ras

def get_fast_max_ras(state, trans_reduc_types, delta_s, delta_0_s, vaccine_limit = 100):
    M = trans_reduc_types
    xst = state[0]
    xit = state[1]
    num_actions = 4
    ds_action = np.zeros((num_actions, 2))
    ds_reward = np.zeros(num_actions)
    i = 0
    yv_bound = [0, np.min([vaccine_limit, xst])]
    yT_bound = [0, M]
    for yv in yv_bound:
        for yT in yT_bound:
            action = np.asarray([yv, yT])
            ds_action[i,:] = action
            ds_reward[i] = get_ras(action, delta_s, delta_0_s)
            i += 1
    max_ras = np.max(ds_reward)
    max_idx = np.argmax(ds_reward)
    return ds_action[max_idx], max_ras


# 3 Large Scale Experiment

In [ ]:
import random
import numpy as np

niter = 10
T = 12 # time horizon
N = 1000 # population size
discretized_N = 100
M = 5 # types of transmission-reducing interventions 
lam = 0.9
discretize = True

act_dim = 2
if discretize == False:
    num_states = (N+1)**2
else:
    discretize_level = np.int(N/discretized_N)
    num_states = (discretized_N+1)**2

# pernalty coefficients
k = np.ones(num_states)*0.001
K = np.ones((num_states, num_states))

# define initial state
xs_init = np.int(N*9/10)
xi_init = N - xs_init
s_init = np.asarray([xs_init, xi_init])

# define reward parameters
unit = 50000
cT_multiplier = N/20 * unit 
wtp = 1 * unit
cost_per_infect = 1 * unit
qaly_loss_per_infect = 1 # QALY
vaccine_price = 0.5 * unit

# define pas paramters 
tau = 0.7
mu = 5
vaccine_limit = N/10

# build heuristic funcion 
V0 = np.zeros(num_states)
for s0 in range(discretized_N+1):
    for s1 in range(discretized_N+1):
        current_s_idx = s0*(discretized_N+1) + s1
        xs = s0*discretize_level
        xi = s1*discretize_level
        s = np.asarray([xs,xi])
        delta_0_s, delta_s = get_ras_coefficients(s, N, M, cT_multiplier, wtp, cost_per_infect, qaly_loss_per_infect, vaccine_price, tau, mu, vaccine_limit)
        _, max_ras = get_fast_max_ras(s, M, delta_s, delta_0_s, vaccine_limit)
        V0[current_s_idx] = max_ras 

## 3.1 Transition probability = bar_pas

### 3.1.1 DRMDP DD1

In [ ]:
V = np.zeros((num_states, T))
xs_table = np.zeros((niter, T))
xi_table = np.zeros((niter, T))
a1_table = np.zeros((niter, T-1))
a2_table = np.zeros((niter, T-1))
for t in range(T-1):
    V[:,t] = V0
episode_rewards = np.zeros(niter)
for i in range(niter):
    print('Episode %d begins' %i)
    total_reward = 0 
    s = s_init
    for t in range(T-1):
        rho_0_s, rho_s = get_bar_pas_coefficients(s, N, M, discretize, discretized_N, tau, mu, vaccine_limit)
        delta_0_s, delta_s = get_ras_coefficients(s, N, M, cT_multiplier, wtp, cost_per_infect, qaly_loss_per_infect, vaccine_price, tau, mu, vaccine_limit)
        ub_a = np.asarray([np.min([s[0], vaccine_limit]), M])
        objective, a = solve_bellman_formulation1(act_dim, num_states, lam, V[:,t+1], k, delta_s, rho_s, delta_0_s, rho_0_s, ub_a)
        # update value table
        if discretize == False:
            current_s_idx = s[0]*(N+1) + s[1]
        else:
            s0 = np.int(s[0]/discretize_level)
            s1 = np.int(s[1]/discretize_level)
            current_s_idx = s0*(discretized_N+1) + s1
        V[current_s_idx,t] = objective
        xs_table[i][t] = s[0]
        xi_table[i][t] = s[1]
        a1_table[i][t] = a[0]
        a2_table[i][t] = a[1]
        # update reward
        total_reward += get_ras(a, delta_s, delta_0_s)
        # sample next state 
        pas = get_bar_pas(a, N, act_dim, rho_0_s, rho_s, discretize, discretized_N)
        next_s_idx = random.choices(np.arange(num_states), pas)[0]
        if discretize == False:
            xs = np.int(next_s_idx/(N+1))
            xi = next_s_idx % (N+1)
        else:
            s0 = np.int(next_s_idx/(discretized_N + 1))
            s1 = next_s_idx % (discretized_N + 1)
            xs = s0*discretize_level
            xi = s1*discretize_level
        next_s = np.asarray([xs, xi])
        s = next_s
    xs_table[i][T-1] = s[0]
    xi_table[i][T-1] = s[1]
    print('Reward', total_reward)
    episode_rewards[i] = total_reward
    print('Episode %d ends' %i)
print('Episodes reward', episode_rewards)
print('Average reward', np.mean(episode_rewards))
print('-std', np.mean(episode_rewards) - 0.5*np.std(episode_rewards))
print('+std', np.mean(episode_rewards) + 0.5*np.std(episode_rewards))
print('XS: \n', xs_table)
print('XI: \n', xi_table)
print('A1: \n', a1_table)
print('A2: \n', a2_table)
print('Average xs', np.mean(xs_table, axis = 0))
print('Average xi', np.mean(xi_table, axis = 0))
print('Average a1', np.mean(a1_table, axis = 0))
print('Average a2', np.mean(a2_table, axis = 0))

### 3.1.2 Regular MDP

In [ ]:
V = np.zeros((num_states, T))
xs_table = np.zeros((niter, T))
xi_table = np.zeros((niter, T))
a1_table = np.zeros((niter, T-1))
a2_table = np.zeros((niter, T-1))
for t in range(T-1):
    V[:,t] = V0
episode_rewards = np.zeros(niter)
for i in range(niter):
    print('Episode %d begins' %i)
    total_reward = 0 
    s = s_init
    for t in range(T-1):
        rho_0_s, rho_s = get_bar_pas_coefficients(s, N, M, discretize, discretized_N, tau, mu, vaccine_limit)
        delta_0_s, delta_s = get_ras_coefficients(s, N, M, cT_multiplier, wtp, cost_per_infect, qaly_loss_per_infect, vaccine_price, tau, mu, vaccine_limit)
        ub_a = np.asarray([np.min([s[0], vaccine_limit]), M])
        objective, a = solve_bellman_formulation3(act_dim, num_states, lam, V[:,t+1], delta_s, rho_s, delta_0_s, rho_0_s, ub_a)
        # update value table
        if discretize == False:
            current_s_idx = s[0]*(N+1) + s[1]
        else:
            s0 = np.int(s[0]/discretize_level)
            s1 = np.int(s[1]/discretize_level)
            current_s_idx = s0*(discretized_N+1) + s1
        V[current_s_idx,t] = objective
        xs_table[i][t] = s[0]
        xi_table[i][t] = s[1]
        a1_table[i][t] = a[0]
        a2_table[i][t] = a[1]
        # update reward
        total_reward += get_ras(a, delta_s, delta_0_s)
        # sample next state 
        pas = get_bar_pas(a, N, act_dim, rho_0_s, rho_s, discretize, discretized_N)
        next_s_idx = random.choices(np.arange(num_states), pas)[0]
        if discretize == False:
            xs = np.int(next_s_idx/(N+1))
            xi = next_s_idx % (N+1)
        else:
            s0 = np.int(next_s_idx/(discretized_N + 1))
            s1 = next_s_idx % (discretized_N + 1)
            xs = s0*discretize_level
            xi = s1*discretize_level
        next_s = np.asarray([xs, xi])
        s = next_s
    xs_table[i][T-1] = s[0]
    xi_table[i][T-1] = s[1]
    print('Reward', total_reward)
    episode_rewards[i] = total_reward
    print('Episode %d ends' %i)
print('Episodes reward', episode_rewards)
print('Average reward', np.mean(episode_rewards))
print('-std', np.mean(episode_rewards) - 0.5*np.std(episode_rewards))
print('+std', np.mean(episode_rewards) + 0.5*np.std(episode_rewards))
print('XS: \n', xs_table)
print('XI: \n', xi_table)
print('A1: \n', a1_table)
print('A2: \n', a2_table)
print('Average xs', np.mean(xs_table, axis = 0))
print('Average xi', np.mean(xi_table, axis = 0))
print('Average a1', np.mean(a1_table, axis = 0))
print('Average a2', np.mean(a2_table, axis = 0))

## 3.2 Transition probability far from bar_pas

### 3.2.1 DRMDP DD1

In [ ]:
V = np.zeros((num_states, T))
xs_table = np.zeros((niter, T))
xi_table = np.zeros((niter, T))
a1_table = np.zeros((niter, T-1))
a2_table = np.zeros((niter, T-1))
for t in range(T-1):
    V[:,t] = V0
episode_rewards = np.zeros(niter)
for i in range(niter):
    print('Episode %d begins' %i)
    total_reward = 0 
    s = s_init
    for t in range(T-1):
        rho_0_s, rho_s = get_bar_pas_coefficients(s, N, M, discretize, discretized_N, tau, mu, vaccine_limit)
        delta_0_s, delta_s = get_ras_coefficients(s, N, M, cT_multiplier, wtp, cost_per_infect, qaly_loss_per_infect, vaccine_price, tau, mu, vaccine_limit)
        ub_a = np.asarray([np.min([s[0], vaccine_limit]), M])
        objective, a = solve_bellman_formulation1(act_dim, num_states, lam, V[:,t+1], k, delta_s, rho_s, delta_0_s, rho_0_s, ub_a)
        # update value table
        if discretize == False:
            current_s_idx = s[0]*(N+1) + s[1]
        else:
            s0 = np.int(s[0]/discretize_level)
            s1 = np.int(s[1]/discretize_level)
            current_s_idx = s0*(discretized_N+1) + s1
        V[current_s_idx,t] = objective
        xs_table[i][t] = s[0]
        xi_table[i][t] = s[1]
        a1_table[i][t] = a[0]
        a2_table[i][t] = a[1]
        # update reward
        total_reward += get_ras(a, delta_s, delta_0_s)
        # sample next state 
        pas = get_far2_from_bar_pas(a, N, act_dim, rho_0_s, rho_s, discretize, discretized_N)
        next_s_idx = random.choices(np.arange(num_states), pas)[0]
        if discretize == False:
            xs = np.int(next_s_idx/(N+1))
            xi = next_s_idx % (N+1)
        else:
            s0 = np.int(next_s_idx/(discretized_N + 1))
            s1 = next_s_idx % (discretized_N + 1)
            xs = s0*discretize_level
            xi = s1*discretize_level
        next_s = np.asarray([xs, xi])
        s = next_s
    xs_table[i][T-1] = s[0]
    xi_table[i][T-1] = s[1]
    print('Reward', total_reward)
    episode_rewards[i] = total_reward
    print('Episode %d ends' %i)
print('Episodes reward', episode_rewards)
print('Average reward', np.mean(episode_rewards))
print('-std', np.mean(episode_rewards) - 0.5*np.std(episode_rewards))
print('+std', np.mean(episode_rewards) + 0.5*np.std(episode_rewards))
print('XS: \n', xs_table)
print('XI: \n', xi_table)
print('A1: \n', a1_table)
print('A2: \n', a2_table)
print('Average xs', np.mean(xs_table, axis = 0))
print('Average xi', np.mean(xi_table, axis = 0))
print('Average a1', np.mean(a1_table, axis = 0))
print('Average a2', np.mean(a2_table, axis = 0))

### 3.2.2 Regular MDP

In [ ]:
V = np.zeros((num_states, T))
xs_table = np.zeros((niter, T))
xi_table = np.zeros((niter, T))
a1_table = np.zeros((niter, T-1))
a2_table = np.zeros((niter, T-1))
for t in range(T-1):
    V[:,t] = V0
episode_rewards = np.zeros(niter)
for i in range(niter):
    print('Episode %d begins' %i)
    total_reward = 0 
    s = s_init
    for t in range(T-1):
        rho_0_s, rho_s = get_bar_pas_coefficients(s, N, M, discretize, discretized_N, tau, mu, vaccine_limit)
        delta_0_s, delta_s = get_ras_coefficients(s, N, M, cT_multiplier, wtp, cost_per_infect, qaly_loss_per_infect, vaccine_price, tau, mu, vaccine_limit)
        ub_a = np.asarray([np.min([s[0], vaccine_limit]), M])
        objective, a = solve_bellman_formulation3(act_dim, num_states, lam, V[:,t+1], delta_s, rho_s, delta_0_s, rho_0_s, ub_a)
        # update value table
        if discretize == False:
            current_s_idx = s[0]*(N+1) + s[1]
        else:
            s0 = np.int(s[0]/discretize_level)
            s1 = np.int(s[1]/discretize_level)
            current_s_idx = s0*(discretized_N+1) + s1
        V[current_s_idx,t] = objective
        xs_table[i][t] = s[0]
        xi_table[i][t] = s[1]
        a1_table[i][t] = a[0]
        a2_table[i][t] = a[1]
        # update reward
        total_reward += get_ras(a, delta_s, delta_0_s)
        # sample next state 
        pas = get_far2_from_bar_pas(a, N, act_dim, rho_0_s, rho_s, discretize, discretized_N)
        next_s_idx = random.choices(np.arange(num_states), pas)[0]
        if discretize == False:
            xs = np.int(next_s_idx/(N+1))
            xi = next_s_idx % (N+1)
        else:
            s0 = np.int(next_s_idx/(discretized_N + 1))
            s1 = next_s_idx % (discretized_N + 1)
            xs = s0*discretize_level
            xi = s1*discretize_level
        next_s = np.asarray([xs, xi])
        s = next_s
    xs_table[i][T-1] = s[0]
    xi_table[i][T-1] = s[1]
    print('Reward', total_reward)
    episode_rewards[i] = total_reward
    print('Episode %d ends' %i)
print('Episodes reward', episode_rewards)
print('Average reward', np.mean(episode_rewards))
print('-std', np.mean(episode_rewards) - 0.5*np.std(episode_rewards))
print('+std', np.mean(episode_rewards) + 0.5*np.std(episode_rewards))
print('XS: \n', xs_table)
print('XI: \n', xi_table)
print('A1: \n', a1_table)
print('A2: \n', a2_table)
print('Average xs', np.mean(xs_table, axis = 0))
print('Average xi', np.mean(xi_table, axis = 0))
print('Average a1', np.mean(a1_table, axis = 0))
print('Average a2', np.mean(a2_table, axis = 0))

# 4 Small Scale Experiment

In [ ]:
import random
import numpy as np

niter = 5
T = 12 # time horizon
N = 200 # population size
discretized_N = 8
M = 5 # types of transmission-reducing interventions 
lam = 0.9
discretize = True

act_dim = 2
if discretize == False:
    num_states = (N+1)**2
else:
    discretize_level = np.int(N/discretized_N)
    num_states = (discretized_N+1)**2

# pernalty coefficients
k = np.ones(num_states) 
K = np.ones((num_states, num_states))


# define initial state
xs_init = np.int(N*7/10)
xi_init = N - xs_init
s_init = np.asarray([xs_init, xi_init])

# define reward parameters 
unit = 50000
cT_multiplier = N/20 * unit
wtp = 1 * unit
cost_per_infect = 1 * unit 
qaly_loss_per_infect = 1
vaccine_price = 0.5 * unit

# define pas paramters 
tau = 0.7
mu = 5

vaccine_limit = N/10

sigma_s = np.zeros(act_dim)
sigma_0_s = 1
bar_Sigma_s = np.ones((num_states, num_states))

# build heuristic funcion 
V0 = np.zeros(num_states)
for s0 in range(discretized_N+1):
    for s1 in range(discretized_N+1):
        current_s_idx = s0*(discretized_N+1) + s1
        xs = s0*discretize_level
        xi = s1*discretize_level
        s = np.asarray([xs,xi])
        delta_0_s, delta_s = get_ras_coefficients(s, N, M, cT_multiplier, wtp, cost_per_infect, qaly_loss_per_infect, vaccine_price, tau, mu, vaccine_limit)
        _, max_ras = get_fast_max_ras(s, M, delta_s, delta_0_s, vaccine_limit)
        V0[current_s_idx] = max_ras 

## 4.1 DRMDP DD1

In [ ]:
V = np.zeros((num_states, T))
xs_table = np.zeros((niter, T))
xi_table = np.zeros((niter, T))
a1_table = np.zeros((niter, T-1))
a2_table = np.zeros((niter, T-1))
for t in range(T-1):
    V[:,t] = V0
episode_rewards = np.zeros(niter)
for i in range(niter):
    print('Episode %d begins' %i)
    total_reward = 0 
    s = s_init
    for t in range(T-1):
        rho_0_s, rho_s = get_bar_pas_coefficients(s, N, M, discretize, discretized_N, tau, mu, vaccine_limit)
        delta_0_s, delta_s = get_ras_coefficients(s, N, M, cT_multiplier, wtp, cost_per_infect, qaly_loss_per_infect, vaccine_price, tau, mu, vaccine_limit)
        ub_a = np.asarray([np.min([s[0], vaccine_limit]), M])
        objective, a = solve_bellman_formulation1(act_dim, num_states, lam, V[:,t+1], k, delta_s, rho_s, delta_0_s, rho_0_s, ub_a)
        # update value table
        if discretize == False:
            current_s_idx = s[0]*(N+1) + s[1]
        else:
            s0 = np.int(s[0]/discretize_level)
            s1 = np.int(s[1]/discretize_level)
            current_s_idx = s0*(discretized_N+1) + s1
        V[current_s_idx,t] = objective
        xs_table[i][t] = s[0]
        xi_table[i][t] = s[1]
        a1_table[i][t] = a[0]
        a2_table[i][t] = a[1]
        # update reward
        total_reward += get_ras(a, delta_s, delta_0_s)
        # sample next state 
        pas = get_far2_from_bar_pas(a, N, act_dim, rho_0_s, rho_s, discretize, discretized_N)
        next_s_idx = random.choices(np.arange(num_states), pas)[0]
        if discretize == False:
            xs = np.int(next_s_idx/(N+1))
            xi = next_s_idx % (N+1)
        else:
            s0 = np.int(next_s_idx/(discretized_N + 1))
            s1 = next_s_idx % (discretized_N + 1)
            xs = s0*discretize_level
            xi = s1*discretize_level
        next_s = np.asarray([xs, xi])
        s = next_s
    xs_table[i][T-1] = s[0]
    xi_table[i][T-1] = s[1]
    print('Reward', total_reward)
    episode_rewards[i] = total_reward
    print('Episode %d ends' %i)
print('Episodes reward', episode_rewards)
print('Average reward', np.mean(episode_rewards))
print('-std', np.mean(episode_rewards) - 0.5*np.std(episode_rewards))
print('+std', np.mean(episode_rewards) + 0.5*np.std(episode_rewards))
print('XS: \n', xs_table)
print('XI: \n', xi_table)
print('A1: \n', a1_table)
print('A2: \n', a2_table)
print('Average xs', np.mean(xs_table, axis = 0))
print('Average xi', np.mean(xi_table, axis = 0))
print('Average a1', np.mean(a1_table, axis = 0))
print('Average a2', np.mean(a2_table, axis = 0))

## 4.2 DRMDP DD2

In [ ]:
V = np.zeros((num_states, T))
xs_table = np.zeros((niter, T))
xi_table = np.zeros((niter, T))
a1_table = np.zeros((niter, T-1))
a2_table = np.zeros((niter, T-1))
for t in range(T-1):
    V[:,t] = V0
episode_rewards = np.zeros(niter)
for i in range(niter):
    print('Episode %d begins' %i)
    total_reward = 0 
    s = s_init
    for t in range(T-1):
        rho_0_s, rho_s = get_bar_pas_coefficients(s, N, M, discretize, discretized_N, tau, mu, vaccine_limit)
        delta_0_s, delta_s = get_ras_coefficients(s, N, M, cT_multiplier, wtp, cost_per_infect, qaly_loss_per_infect, vaccine_price, tau, mu, vaccine_limit)
        ub_a = np.asarray([np.min([s[0], vaccine_limit]), M])
        objective, a = solve_bellman_formulation2(act_dim, num_states, lam, V[:,t+1], k, K, delta_s, rho_s, sigma_s, delta_0_s, 
                                                  rho_0_s, sigma_0_s, bar_Sigma_s, ub_a)
        # update value table
        if discretize == False:
            current_s_idx = s[0]*(N+1) + s[1]
        else:
            s0 = np.int(s[0]/discretize_level)
            s1 = np.int(s[1]/discretize_level)
            current_s_idx = s0*(discretized_N+1) + s1
        V[current_s_idx,t] = objective
        xs_table[i][t] = s[0]
        xi_table[i][t] = s[1]
        a1_table[i][t] = a[0]
        a2_table[i][t] = a[1]
        # update reward
        total_reward += get_ras(a, delta_s, delta_0_s)
        # sample next state 
        pas = get_far2_from_bar_pas(a, N, act_dim, rho_0_s, rho_s, discretize, discretized_N)
        next_s_idx = random.choices(np.arange(num_states), pas)[0]
        if discretize == False:
            xs = np.int(next_s_idx/(N+1))
            xi = next_s_idx % (N+1)
        else:
            s0 = np.int(next_s_idx/(discretized_N + 1))
            s1 = next_s_idx % (discretized_N + 1)
            xs = s0*discretize_level
            xi = s1*discretize_level
        next_s = np.asarray([xs, xi])
        s = next_s
    xs_table[i][T-1] = s[0]
    xi_table[i][T-1] = s[1]
    print('Reward', total_reward)
    episode_rewards[i] = total_reward
    print('Episode %d ends' %i)
print('Episodes reward', episode_rewards)
print('Average reward', np.mean(episode_rewards))
print('-std', np.mean(episode_rewards) - 0.5*np.std(episode_rewards))
print('+std', np.mean(episode_rewards) + 0.5*np.std(episode_rewards))
print('XS: \n', xs_table)
print('XI: \n', xi_table)
print('A1: \n', a1_table)
print('A2: \n', a2_table)
print('Average xs', np.mean(xs_table, axis = 0))
print('Average xi', np.mean(xi_table, axis = 0))
print('Average a1', np.mean(a1_table, axis = 0))
print('Average a2', np.mean(a2_table, axis = 0))